<a href="https://colab.research.google.com/github/Rajatava/Signal-Translation/blob/master/vae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


In [11]:
import numpy as np
from scipy import signal
from scipy.io import wavfile 
# Load the Drive helper and mount


# This will prompt for authorization.
drive.mount('/content/drive')

# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive"

samplerate, samples = wavfile.read('/content/drive/My Drive/voice.wav')
frequencies, times, spectrogram = signal.stft(samples, samplerate)
s = spectrogram.shape
print(s)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 106106213
 3rdsem_Bill.PDF
 4thsem_Bill.PDF
 A20006906554.pdf
 AAMydoc
 Access.gdoc
'Arduino & Python Workshop- 2-09-2018.pdf'
'Colab Notebooks'
 Contacts.vcf
'Copy of SetupSTM32CubeMX-5.4.0.linux'
'DewarpNet Review.docx'
'ENGLISH  PAPER.docx'
 English.txt
 file.pdf
 GeeksforGeeks.rar
'Getting started.pdf'
'Gmail - A Request for giving NOC on behalf of College as the response of Internship offer letter from Praesus Technologies Private  Ltd.PDF'
' guitar.wav'
'Internship Letter Official_Rajatava_Mukherje-signed.PDF'
'New Doc 2018-11-03 12.26.47.pdf'
 nonRepeatingCharacter.cpp
 old_user.py.gdoc
'RajatavaMukherjee(2ndyr-A-81)MarDoc_NptelCourse(JOCusingPython).PDF'
'Raj on fire.mp4'
'Resume (1).PDF'
'Resume of Rajatava Mukherjee.docx'
 Resume.PDF
 Resume-template.gdoc
'Sanet Review github.doc'
'Spelling Reform.odt'
'Untitled document.gdoc'
'Untitled form.gform'

In [12]:
!pip install tensorflow==2.0.0-rc0
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os
import tensorflow
from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

In [0]:
# reparameterization trick
# instead of sampling from Q(z|X), sample epsilon = N(0,I)
# z = z_mean + sqrt(var) * epsilon
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


sd = s[0]*s[1]

sz = np.reshape(spectrogram,sd)
original_dim = sd


In [34]:
original_dim

122034

In [0]:
# network parameters
input_shape = (original_dim,)
intermediate_dim = int(sd/50)
batch_size = 128
latent_dim = int(sd/10)
epochs = 50

In [36]:
print(input_shape,intermediate_dim,latent_dim)

(122034,) 2440 12203


In [37]:
# VAE model = encoder + decoder


# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim)(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()


# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim)(latent_inputs)
outputs = Dense(original_dim)(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')



Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 122034)]     0                                            
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 2440)         297765400   encoder_input[0][0]              
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 12203)        29787523    dense_7[0][0]                    
__________________________________________________________________________________________________
z_log_var (Dense)               (None, 12203)        29787523    dense_7[0][0]                    
____________________________________________________________________________________________

In [39]:
models = (encoder, decoder)
data = (sz)

# VAE loss = mse_loss or xent_loss + kl_loss

reconstruction_loss = mse(inputs, outputs)
reconstruction_loss = binary_crossentropy(inputs,outputs)

reconstruction_loss *= original_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
vae.summary()


    

Model: "vae_mlp"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 122034)]     0                                            
__________________________________________________________________________________________________
encoder (Model)                 [(None, 12203), (Non 357340446   encoder_input[0][0]              
__________________________________________________________________________________________________
decoder (Model)                 (None, 122034)       327662754   encoder[1][2]                    
__________________________________________________________________________________________________
tf_op_layer_clip_by_value/Minim [(None, 122034)]     0           decoder[1][0]                    
____________________________________________________________________________________________

In [0]:
np.transpose(sz).shape

(1704219,)

In [40]:
vae.fit(sz,
        epochs=epochs,
        batch_size=batch_size)
  


ValueError: ignored